In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.cluster import OPTICS

In [ ]:
import numpy as np; 
# Data manipulation
# ==============================================================================
import numpy as np
import pandas as pd

# Plots
# ==============================================================================
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
plt.style.use('fivethirtyeight')

# Modelado y Forecasting
# ==============================================================================
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregMultiOutput import ForecasterAutoregMultiOutput
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster

# Warnings configuration
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

In [ ]:
dataset = pd.read_csv('table_orders.csv') # turned into parallel hexagon array
dataset.head()

In [ ]:
dataset =  dataset.set_index('time')

In [ ]:
data = dataset

In [ ]:
data = data.iloc[0:1000,0:1000]
data.shape

In [ ]:
data = data.iloc[0:100,0:100]

In [ ]:
table11 = data

In [ ]:
data

In [ ]:
dataset4 = pd.read_csv('timedddd.csv',sep = ";") # turned into parallel hexagon array
#dataset4 =  dataset.set_index('time')
dataset4.head()
dataset4 =  dataset4.set_index('time')

In [ ]:
dataset4 =  dataset4.set_index('time')

# XGBRegressorAutoreg

In [ ]:
d = [[]]

data = data.astype(int)
end_validation = 976
for index in range(dataset4.shape[1]):
    columnSeriesObj = dataset4.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoreg(
                regressor = XGBRegressor(objective='reg:squarederror', n_estimators=1000),
                lags      = 24
             )
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500],
              'max_depth': [4, 6]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj)),#data[:,index],table11 = pd.Series(list(table1))
                        exog        = columnSeriesObj,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 24,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 976,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 24,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
   
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();
    d = predictions
    print(predictions.index)

# RandomForestRegressorMultiOutput

In [ ]:
data = data.iloc[0:1000,0:20]

In [ ]:
data = dataset4

In [ ]:
data = data.astype(int)
end_validation = 900
for index in range(data.shape[1]):
    columnSeriesObj = data.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoregMultiOutput(
                regressor = RandomForestRegressor(max_depth=4),
                steps     = 24, #60
                lags      = 24 # This value will be replaced in the grid search
             )
# Regressor's hyperparameters
    columnSeriesObj1 = columnSeriesObj[24:1000]
    columnSeriesObj2 = columnSeriesObj[0:976]
    columnSeriesObj4 = columnSeriesObj[0:976]

    param_grid = {'n_estimators': [100, 500],
              'max_depth': [4, 6]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj1)),#data[:,index],table11 = pd.Series(list(table1))
                        exog        = columnSeriesObj2,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 24,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 800,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj2)),
                            initial_train_size = len(columnSeriesObj2[:end_validation]),
                            steps      = 24,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();
    print(y)

# LGBMRegressor+AutoregMultiOutput

In [ ]:
data = data.astype(int)
end_validation = 80
for index in range(data.shape[1]):
    columnSeriesObj = data.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoregMultiOutput(
                regressor = LGBMRegressor(max_depth=8),
                steps     = 20, #60
                lags      = 20 # This value will be replaced in the grid search
             )
    columnSeriesObj1 = columnSeriesObj[24:1000]
    columnSeriesObj2 = columnSeriesObj[0:976]
    
    
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500],
              'max_depth': [4, 6]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj2)),#data[:,index],table11 = pd.Series(list(table1))
                        exog        = columnSeriesObj1,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 20,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 80,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 24,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();

# LGBMRegressor+AutoregMultiOutput

In [ ]:
data = data.astype(int)
end_validation = 80
for index in range(data.shape[1]):
    columnSeriesObj = data.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoregMultiOutput(
                regressor = LGBMRegressor(max_depth=8),
                steps     = 20, #60
                lags      = 20 # This value will be replaced in the grid search
             )
    
    columnSeriesObj1 = columnSeriesObj[24:1000]
    columnSeriesObj2 = columnSeriesObj[0:976]
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500],
              'max_depth': [4, 6]}
# Lags used as predictors
    lags_grid = [[1,24], [1,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj2)),#data[:,index],table11 = pd.Series(list(table1))
                        exog        = columnSeriesObj1,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 20,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 80,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 24,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.datasets import make_regression

# LGBMAutoregMultiOutput

In [ ]:
data = data.astype(int)
end_validation = 80
for index in range(data.shape[1]):
    columnSeriesObj = data.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoregMultiOutput(
                regressor = LGBMRegressor(n_estimators=100, max_depth=8),
                steps     = 20, #60
                lags      = 20 # This value will be replaced in the grid search
             )
# Regressor's hyperparameters
    columnSeriesObj1 = columnSeriesObj[24:1000]
    columnSeriesObj2 = columnSeriesObj[0:976]

    param_grid = {'n_estimators': [100, 500],
              'max_depth': [4, 6]}
# Lags used as predictors
    lags_grid = [[1,24], [1,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj)),#data[:,index],table11 = pd.Series(list(table1))
                        exog        = columnSeriesObj,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 20,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 80,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 24,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();

In [ ]:
data = data.astype(int)
end_validation = 60
for index in range(data.shape[1]):
    columnSeriesObj = data.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoregMultiOutput(
                regressor = LGBMRegressor(n_estimators=100),
                steps     = 20, #60
                lags      = [1, 2, 3, 23, 24, 25, 47, 48, 49], # This value will be replaced in the grid search
             )
    columnSeriesObj1 = columnSeriesObj[24:1000]
    columnSeriesObj2 = columnSeriesObj[0:976]
    
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500],
              'max_depth': [4, 21]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj2)),#data[:,index],table11 = pd.Series(list(table1))
                        exog        = columnSeriesObj1,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 20,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 80,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 10,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();

In [ ]:
data = data.astype(int)
end_validation = 60
for index in range(data.shape[1]):
    columnSeriesObj = data.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoregMultiOutput(
                regressor = LGBMRegressor(n_estimators=100, max_depth=8),
                steps     = 20, #60
                lags      = 20 # This value will be replaced in the grid search
             )
    
    columnSeriesObj1 = columnSeriesObj[24:1000]
    columnSeriesObj2 = columnSeriesObj[0:976]
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500],
              'max_depth': [4, 6]}
# Lags used as predictors
    lags_grid = [[1,24], [1,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj1)),#data[:,index],table11 = pd.Series(list(table1))
                        exog        = columnSeriesObj2,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 20,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 80,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 10,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();

In [ ]:

# forecast monthly births with xgboost
from numpy import asarray
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from matplotlib import pyplot

In [ ]:
data.shape

# XGBRegressor

In [ ]:
data = data.astype(int)
end_validation = 80
for index in range(data.shape[1]):
    columnSeriesObj = data.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoreg(
                regressor = XGBRegressor(objective='reg:squarederror', n_estimators=1000),
                lags      = 24
             )
    
    columnSeriesObj1 = columnSeriesObj[24:1000]
    columnSeriesObj2 = columnSeriesObj[0:976]
    
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500],
              'max_depth': [4, 6]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj1)),#data[:,index],table11 = pd.Series(list(table1))
                        exog        = columnSeriesObj2,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 20,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 80,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 10,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
    print(index)
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();

# XGBRegressor

In [ ]:
data = data.astype(int)
end_validation = 80
for index in range(data.shape[1]):
    columnSeriesObj = data.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoreg(
                regressor = XGBRegressor(objective='reg:squarederror', n_estimators=1000),
                lags      = 24
             )
    columnSeriesObj1 = columnSeriesObj[24:1000]
    columnSeriesObj2 = columnSeriesObj[0:976]
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500],
              'max_depth': [4, 6]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj2)),#data[:,index],table11 = pd.Series(list(table1))
                        #exog        = columnSeriesObj1,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 10,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 80,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 10,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
   
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();
    
    print(predictions.index)

# AutoregXGBRegressor

In [ ]:
d = [[]]

data = data.astype(int)
end_validation = 80
for index in range(data.shape[1]):
    columnSeriesObj = data.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoreg(
                regressor = XGBRegressor(objective='reg:squarederror', n_estimators=1000),
                lags      = 24
             )
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500],
              'max_depth': [4, 6]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj)),#data[:,index],table11 = pd.Series(list(table1))
                        exog        = columnSeriesObj,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 10,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 80,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 10,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
   
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();
    d = predictions
    print(predictions.index)

# AutoregXGBRegressor

In [ ]:
d = [[]]

data = data.astype(int)
end_validation = 80
for index in range(data.shape[1]):
    columnSeriesObj = data.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoreg(
                regressor = XGBRegressor(objective='reg:squarederror', n_estimators=1000),
                lags      = 24
             )
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500],
              'max_depth': [4, 6]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj)),#data[:,index],table11 = pd.Series(list(table1))
                        exog        = columnSeriesObj,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 10,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 80,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 10,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
   
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();
    d = predictions
    print(predictions.index)

In [ ]:

# forecast monthly births with xgboost
from numpy import asarray
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import AdaBoostRegressor
from matplotlib import pyplot

In [ ]:
data

# AdaBoostRegressor

In [ ]:
d = [[]]

data = data.astype(int)
end_validation = 80
for index in range(data.shape[1]):
    columnSeriesObj = data.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoreg(
                regressor = AdaBoostRegressor(),
                lags      = 24
             )
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj)),#data[:,index],table11 = pd.Series(list(table1))
                        exog        = columnSeriesObj,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 10,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 80,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 10,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
   
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();
    d = predictions
    print(predictions.index)

In [ ]:
dataset = dataset.iloc[0:1000,0:1000]


# AdaBoostRegressor

In [ ]:
d = [[]]

data = data.astype(int)
end_validation = 800
for index in range(dataset.shape[1]):
    columnSeriesObj = dataset.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoreg(
                regressor = AdaBoostRegressor(),
                lags      = 24
             )
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj)),#data[:,index],table11 = pd.Series(list(table1))
                        exog        = columnSeriesObj,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 24,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 976,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 10,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
   
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();
    d = predictions
    print(predictions.index)

In [ ]:
dataset = pd.read_csv('datadddd.csv',sep = ";") # turned into parallel hexagon array
dataset.head()


In [ ]:
datasetttt =  dataset.set_index('time')

In [ ]:
d = [[]]

#data = data.astype(int)
end_validation = 600
for index in range(datasetttt.shape[1]):
    columnSeriesObj = datasetttt.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoreg(
                regressor = AdaBoostRegressor(),
                lags      = 24
             )
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj)),#data[:,index],table11 = pd.Series(list(table1))
                        exog        = columnSeriesObj,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 24,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 976,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 10,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
   
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();
    d = predictions
    print(predictions.index)

In [ ]:
d = [[]]

#data = data.astype(int)
end_validation = 976
for index in range(datasetttt.shape[1]):
    columnSeriesObj = datasetttt.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoreg(
                regressor = AdaBoostRegressor(),
                lags      = 24
             )
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj)),#data[:,index],table11 = pd.Series(list(table1))
                        exog        = columnSeriesObj,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 24,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 976,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 10,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
   
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();
    d = predictions
    print(predictions.index)

# RandomForestRegressor

In [ ]:
data = data.astype(int)
end_validation = 80
for index in range(datasetttt.shape[1]):
    columnSeriesObj = datasetttt.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoregMultiOutput(
                regressor = RandomForestRegressor(max_depth=14),
                steps     = 24, #60
                lags      = 20 # This value will be replaced in the grid search
             )
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500],
              'max_depth': [4, 6]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj)),#data[:,index],table11 = pd.Series(list(table1))
                        exog        = columnSeriesObj,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 24,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 80,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 24,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();

# AutoregXGBRegressor

In [ ]:
d = [[]]

data = data.astype(int)
end_validation = 976
for index in range(datasetttt.shape[1]):
    columnSeriesObj = datasetttt.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoreg(
                regressor = XGBRegressor(objective='reg:squarederror', n_estimators=1000),
                lags      = 24
             )
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500],
              'max_depth': [4, 6]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj)),#data[:,index],table11 = pd.Series(list(table1))
                        exog        = columnSeriesObj,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 10,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 976,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 10,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
   
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();
    d = predictions
    print(predictions.index)

# LGBMRegressor

In [ ]:
data = data.astype(int)
end_validation = 976
for index in range(datasetttt.shape[1]):
    columnSeriesObj = datasetttt.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoregMultiOutput(
                regressor = LGBMRegressor(max_depth=8),
                steps     = 24, #60
                lags      = 20 # This value will be replaced in the grid search
             )
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500],
              'max_depth': [4, 6]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj)),#data[:,index],table11 = pd.Series(list(table1))
                        exog        = columnSeriesObj,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 24,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 976,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 24,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();

In [ ]:
data = data.astype(int)
end_validation = 976
for index in range(datasetttt.shape[1]):
    columnSeriesObj = datasetttt.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoregMultiOutput(
                regressor = RandomForestRegressor(max_depth=4),
                steps     = 24, #60
                lags      = 1 # This value will be replaced in the grid search
             )
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500],
              'max_depth': [4, 6]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj)),#data[:,index],table11 = pd.Series(list(table1))
                        exog        = columnSeriesObj,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 24,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 976,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 24,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    
    ax.legend();

In [ ]:
data = data.astype(int)
end_validation = 976
for index in range(datasetttt.shape[1]):
    columnSeriesObj = datasetttt.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoregMultiOutput(
                regressor = RandomForestRegressor(max_depth=16),
                steps     = 24, #60
                lags      = 1 # This value will be replaced in the grid search
             )
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500],
              'max_depth': [4, 6]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj)),#data[:,index],table11 = pd.Series(list(table1))
                        exog        = columnSeriesObj,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 24,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 976,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 24,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    
    
    ax.legend();

In [ ]:
dataset4 =  dataset4.set_index('time')

# LGBMRegressor

In [ ]:
data = data.astype(int)
d = dataset
#d = 0
d = np.asarray(d)
end_validation = 952
for index in range(dataset4.shape[1]):
    columnSeriesObj = dataset4.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoregMultiOutput(
                regressor = LGBMRegressor(max_depth=8),
                steps     = 24, #60
                lags      = 20 # This value will be replaced in the grid search
             )
    columnSeriesObj1 = columnSeriesObj[24:1000]
    columnSeriesObj2 = columnSeriesObj[0:976]
    columnSeriesObj4 = columnSeriesObj[0:976]
    #columnSeriesOb1 = columnSeriesObj.shift(periods=24)
    #columnSeriesOb2 = columnSeriesOb1.fillna(0) 
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500],
              'max_depth': [4, 6]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj2)),#data[:,index],table11 = pd.Series(list(table1))
                        exog        = columnSeriesObj1,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 24,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 952,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj4)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 24,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();
    #d[index, :] = d[index, :] + predictions

In [ ]:
predictions.shape

#  AutoregRandomForestRegressor

In [ ]:
data = data.astype(int)
d = datasetttt
#d = 0
d = np.asarray(d)
end_validation = 1100
for index in range(datasetttt.shape[1]):
    columnSeriesObj = datasetttt.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoreg(
                regressor = LGBMRegressor(max_depth=8),
                #steps     = 24, #60
                lags      = 20 # This value will be replaced in the grid search
             )
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500],
              'max_depth': [4, 6]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj)),#data[:,index],table11 = pd.Series(list(table1))
                        #exog        = columnSeriesObj,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 24,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 1100,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 24,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();
    #d[index, :] = d[index, :] + predictions

In [ ]:
predictions

#  AutoregXGBRegressor

In [ ]:
data = data.astype(int)
d = datasetttt
#d = 0
d = np.asarray(d)
end_validation = 1100
for index in range(datasetttt.shape[1]):
    columnSeriesObj = datasetttt.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoreg(
                regressor = XGBRegressor(objective='reg:squarederror', n_estimators=1000),
                #steps     = 24, #60
                lags      = 20 # This value will be replaced in the grid search
             )
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500],
              'max_depth': [4, 6]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj)),#data[:,index],table11 = pd.Series(list(table1))
                        #exog        = columnSeriesObj,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 24,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 1100,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 24,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();
    #d[index, :] = d[index, :] + predictions

In [ ]:
numpy.roll(a,24)

In [ ]:
import numpy

# AutoregLGBMRegressor X

In [ ]:
data = data.astype(int)
d = datasetttt
#d = 0
d = np.asarray(d)
end_validation = 1100
for index in range(datasetttt.shape[1]):
    columnSeriesObj = datasetttt.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoregMultiOutput(
                regressor = LGBMRegressor(max_depth=8),
                steps     = 24, #60
                lags      = 20 # This value will be replaced in the grid search
             )
    
    columnSeriesOb1 = columnSeriesObj.shift(periods=24)
    columnSeriesOb2 = columnSeriesOb1.fillna(0)    
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500],
              'max_depth': [4, 6]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj)),#data[:,index],table11 = pd.Series(list(table1))   
                        exog        = columnSeriesOb2,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 24,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 900,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 24,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();
    #d[index, :] = d[index, :] + predictions

# RandomForestRegressorAutoreg

In [ ]:
data = data.astype(int)
d = datasetttt
#d = 0
d = np.asarray(d)
end_validation = 1100
for index in range(datasetttt.shape[1]):
    columnSeriesObj = datasetttt.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoreg(
                regressor = RandomForestRegressor(max_depth=16),
                #steps     = 24, #60
                lags      = 20 # This value will be replaced in the grid search
             )
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500],
              'max_depth': [4, 6]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj)),#data[:,index],table11 = pd.Series(list(table1))
                        #exog        = columnSeriesObj,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 24,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 1100,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 24,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();
    #d[index, :] = d[index, :] + predictions

In [ ]:
predictions

#  UNIIVARIATE

# AutoregAdaBoostRegressor

In [ ]:
data = data.astype(int)
d = datasetttt
#d = 0
d = np.asarray(d)
end_validation = 1100
for index in range(datasetttt.shape[1]):
    columnSeriesObj = datasetttt.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoreg(
                regressor = AdaBoostRegressor(),
                #steps     = 24, #60
                lags      = 20 # This value will be replaced in the grid search
             )
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj)),#data[:,index],table11 = pd.Series(list(table1))
                        #exog        = columnSeriesObj,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 24,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 1100,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 24,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();
    #d[index, :] = d[index, :] + predictions

# AutoregMultiOutput XGBRegressor

In [ ]:
data = data.astype(int)
d = dataset
#d = 0
d = np.asarray(d)
end_validation = 952
for index in range(dataset4.shape[1]):
    columnSeriesObj = dataset4.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoregMultiOutput(
                regressor = XGBRegressor(objective='reg:squarederror', n_estimators=1000),
                steps     = 24, #60
                lags      = 20 # This value will be replaced in the grid search
             )
    columnSeriesObj1 = columnSeriesObj[24:1000]
    columnSeriesObj2 = columnSeriesObj[0:976]
    columnSeriesObj4 = columnSeriesObj[0:976]
    #columnSeriesOb1 = columnSeriesObj.shift(periods=24)
    #columnSeriesOb2 = columnSeriesOb1.fillna(0) 
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500],
              'max_depth': [4, 6]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj2)),#data[:,index],table11 = pd.Series(list(table1))
                        exog        = columnSeriesObj1,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 24,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 952,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj4)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 24,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();
    #d[index, :] = d[index, :] + predictions

In [ ]:
RandomForestRegressor(max_depth=4)

# AutoregMultiOutputRandomForestRegressor

In [ ]:
#data = data.astype(int)
#d = datasetttt
#d = 0
#d = np.asarray(d)
end_validation = 900
for index in range(dataset4.shape[1]):
    columnSeriesObj = dataset4.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoregMultiOutput(
                regressor = RandomForestRegressor(max_depth=14),
                steps     = 24, #60
                lags      = 20 # This value will be replaced in the grid search
             )
    
    columnSeriesObj1 = columnSeriesObj[24:1000]
    columnSeriesObj2 = columnSeriesObj[0:976]
    columnSeriesObj4 = columnSeriesObj[0:976]    
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500],
              'max_depth': [4, 6]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj2)),#data[:,index],table11 = pd.Series(list(table1))   
                        exog        = columnSeriesObj1,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 24,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 900,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 24,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();
    #d[index, :] = d[index, :] + predictions

# AutoregMultiOutputAdaBoostRegressor

In [ ]:
#data = data.astype(int)
#d = #datasetttt
#d = 0
#d = np.asarray(d)
end_validation = 900
for index in range(dataset4.shape[1]):
    columnSeriesObj = dataset4.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoregMultiOutput(
                regressor = AdaBoostRegressor(),
                steps     = 24, #60
                lags      = 20 # This value will be replaced in the grid search
             )
    
    columnSeriesObj1 = columnSeriesObj[24:1000]
    columnSeriesObj2 = columnSeriesObj[0:976]
    columnSeriesObj4 = columnSeriesObj[0:976]    
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj2)),#data[:,index],table11 = pd.Series(list(table1))   
                        exog        = columnSeriesObj1,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 24,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 900,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 24,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();
    #d[index, :] = d[index, :] + predictions

In [ ]:
dataset = data.iloc[0:1000,0:1000]
dataset.shape

In [ ]:
#data = data.astype(int)
#d = #datasetttt
#d = 0
#d = np.asarray(d)
end_validation = 900
for index in range(dataset4.shape[1]):
    columnSeriesObj = dataset4.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoregMultiOutput(
                regressor = AdaBoostRegressor(),
                steps     = 24, #60
                lags      = 20 # This value will be replaced in the grid search
             )
    
    columnSeriesObj1 = columnSeriesObj[24:1000]
    columnSeriesObj2 = columnSeriesObj[0:976]
    columnSeriesObj4 = columnSeriesObj[0:976]    
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj2)),#data[:,index],table11 = pd.Series(list(table1))   
                        exog        = columnSeriesObj1,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 24,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 900,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 24,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();
    #d[index, :] = d[index, :] + predictions

In [ ]:
dataset4

In [ ]:
# Hyperparameter Grid search
# ==============================================================================
end_validation = 1000
end_train = 1000
for index in range(dataset4.shape[1]):
    columnSeriesObj = dataset4.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))

    forecaster = ForecasterAutoreg(
                    regressor = make_pipeline(StandardScaler(), Ridge()),
                    lags      = 24 # This value will be replaced in the grid search
              )

# Regressor's hyperparameters
    param_grid = {'ridge__alpha': np.logspace(-3, 5, 10)}

# Lags used as predictors
    lags_grid = [5, 24, [1, 2, 3, 23, 24, 25, 47, 48, 49]]

    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj)),
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 24,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = len(data[:end_train]),
                        return_best = True,
                        verbose     = False
                  )
    # Backtest
# ==============================================================================
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 24,
                            metric     = 'mean_absolute_error',
                            refit = False,
                            verbose    = True
                        )
    fig, ax = plt.subplots(figsize=(12, 8))
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();
    #d[index, :] = d[index, :] + predictions

In [ ]:
datasetttt = dataset4

In [ ]:
data = data.astype(int)
d = datasetttt
#d = 0
d = np.asarray(d)
end_validation = 1100
for index in range(datasetttt.shape[1]):
    columnSeriesObj = datasetttt.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoreg(
                regressor = XGBRegressor(objective='reg:squarederror', n_estimators=1000),
                #steps     = 24, #60
                lags      = 20 # This value will be replaced in the grid search
             )
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500],
              'max_depth': [4, 6]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj)),#data[:,index],table11 = pd.Series(list(table1))
                        #exog        = columnSeriesObj,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 24,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 1100,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 24,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();
    #d[index, :] = d[index, :] + predictions

In [ ]:
data = data.astype(int)
d = datasetttt
#d = 0
d = np.asarray(d)
end_validation = 1100
for index in range(datasetttt.shape[1]):
    columnSeriesObj = datasetttt.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoreg(
                regressor = RandomForestRegressor(max_depth=16),
                #steps     = 24, #60
                lags      = 20 # This value will be replaced in the grid search
             )
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500],
              'max_depth': [4, 6]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj)),#data[:,index],table11 = pd.Series(list(table1))
                        #exog        = columnSeriesObj,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 24,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 1100,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 24,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();
    #d[index, :] = d[index, :] + predictions

In [ ]:
hour_init_prediction = 11

In [ ]:
#data = data.astype(int)
#d = #datasetttt
#d = 0
#d = np.asarray(d)
end_validation = 900
for index in range(dataset4.shape[1]):
    columnSeriesObj = dataset4.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoregMultiOutput(
                regressor = AdaBoostRegressor(),
                steps     = 24, #60
                lags      = 20 # This value will be replaced in the grid search
             )
    
    columnSeriesObj1 = columnSeriesObj[24:1000]
    columnSeriesObj2 = columnSeriesObj[0:976]
    columnSeriesObj4 = columnSeriesObj[0:976]    
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj2)),#data[:,index],table11 = pd.Series(list(table1))   
                        exog        = columnSeriesObj1,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 24,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 900,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 24,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)

#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();
    #d[index, :] = d[index, :] + predictions

# ForecasterAutoregMultiOutput

In [ ]:
#data = data.astype(int)
#d = #datasetttt
#d = 0
#d = np.asarray(d)
end_validation = 900
for index in range(dataset4.shape[1]):
    columnSeriesObj = dataset4.iloc[:, index]
    columnSeriesObj = pd.Series(list(columnSeriesObj))
    forecaster = ForecasterAutoregMultiOutput(
                regressor = AdaBoostRegressor(),
                steps     = 24, #60
                lags      = 20 # This value will be replaced in the grid search
             )
    
    columnSeriesObj1 = columnSeriesObj[24:1000]
    columnSeriesObj2 = columnSeriesObj[0:976]
    columnSeriesObj4 = columnSeriesObj[0:976]    
# Regressor's hyperparameters
    param_grid = {'n_estimators': [100, 500]}
# Lags used as predictors
    lags_grid = [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24], [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
    results_grid = grid_search_forecaster(
                        forecaster  = forecaster,
                        y           = pd.Series(list(columnSeriesObj2)),#data[:,index],table11 = pd.Series(list(table1))   
                        exog        = columnSeriesObj1,#data[:,index],
                        param_grid  = param_grid,
                        lags_grid   = lags_grid,
                        steps       = 24,
                        metric      = 'mean_absolute_error',
                        refit       = False,
                        initial_train_size = 900,
                        return_best = True,
                        verbose     = False
                  )
    metric, predictions = backtesting_forecaster(
                            forecaster = forecaster,
                            y          = pd.Series(list(columnSeriesObj)),
                            initial_train_size = len(columnSeriesObj[:end_validation]),
                            steps      = 24,
                            metric     = 'mean_absolute_error',
                            refit      = False,
                            verbose    = False)
    
#for index in range(data.shape[1]):
    #columnSeriesObj = data.iloc[:, index]
    #columnSeriesObj = pd.Series(list(columnSeriesObj))                      
    fig, ax = plt.subplots(figsize=(12, 8))
    columnSeriesObj.iloc[predictions.index].plot(linewidth=2, label='real', ax=ax)
    predictions.plot(linewidth=2, label='prediction', ax=ax)
    ax.set_title('Prediction vs real orders')
    ax.legend();
    #d[index, :] = d[index, :] + predictions